In [42]:
import numpy as np 
import pandas as pd
import string
import nltk
import re

In [2]:
df = pd.read_csv("training_twitter_x_y_train.csv")

In [8]:
df.loc[0,"text"]

'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

In [18]:
nltk.download('stopwords')
stop = stopwords.words('english')
punc = [a for a in string.punctuation]
stop += punc
# print(stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'stopwords' is not defined

In [24]:
# print(stop)

In [161]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [165]:
def give_words(tweet):
    clean = re.sub(r"""[,.;@#?!&$]+\ *"""," ",tweet, flags=re.VERBOSE)
    words = clean.strip().split(" ")
    return words

In [177]:
def clean_words(words):
    output_words = []
    for w in words:
        if w == '':
            continue
        if w.lower() not in stop:
            pos = pos_tag([w])                                 
            clean_word = lemmatizer.lemmatize(w, get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [179]:
data = []
for i in range(df.shape[0]):
    text = df.at[i,"text"]
    words = give_words(text)
    output_words = clean_words(words)
    data.append([output_words,df.at[i,"airline_sentiment"]])
    

In [183]:
x_train = [" ".join(a[0]) for a in data]
y_train = [a[1] for a in data]

In [185]:
df_test = pd.read_csv("test_twitter_x_test.csv")

In [190]:
data_test = []
for i in range(df_test.shape[0]):
    text = df_test.at[i,"text"]
    words = give_words(text)
    output_words = clean_words(words)
    data_test.append(output_words)

In [191]:
x_test = [" ".join(a) for a in data_test]

In [199]:
count_vec = CountVectorizer(max_features = 2000)
x_train_features = count_vec.fit_transform(x_train)

In [202]:
x_test_features = count_vec.transform(x_test)

In [203]:
from sklearn import naive_bayes

In [204]:
gnb = naive_bayes.GaussianNB()

In [207]:
gnb.fit(x_train_features.todense(),y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [208]:
y_pred = gnb.predict(x_test_features.todense())

In [213]:
import csv
np.savetxt("predictions.csv", y_pred, delimiter=",", fmt='%s')

In [214]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()

In [215]:
rf.fit(x_train_features.todense(),y_train)
y_pred2 = rf.predict(x_test_features.todense())

In [216]:
import csv
np.savetxt("predictions2.csv", y_pred2, delimiter=",", fmt='%s')